In [1]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
!pip install geopy
import json 

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
!pip install folium
import folium

import string

print('Libraries imported.')

Solving environment: failed

InvalidVersionSpecError: Invalid version spec: =2.7

Libraries imported.


#### We import the Boroughs, Area, Population and coordinates of the city of London from Wikipedia

In [2]:
URL = 'https://en.wikipedia.org/wiki/List_of_London_boroughs'
dfs=pd.read_html(URL)
#save a dataframe with the table
boroughs=dfs[0]
boroughs.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2019 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,212906,".mw-parser-output .geo-default,.mw-parser-outp...",25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,395896,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,248287,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,329771,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,332336,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20


In [3]:
#I rename the columns so they are easier to handle
boroughs.rename(columns={'Co-ordinates':'Coordinates','Population (2019 est)[1]':'Population','Area (sq mi)':'Area'},inplace=True)
boroughs.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area,Population,Coordinates,Nr. in map
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,212906,".mw-parser-output .geo-default,.mw-parser-outp...",25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,395896,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,248287,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,329771,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,332336,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20


Now let's create the dataframe that will have the Borough name, population, coordinates (splitted in longitude and latitude), Area and the measure 'Concentration'

In [4]:
#We select the columns from the dataframe boroughs that are relevant to us
df= boroughs[["Borough","Population","Coordinates","Area"]]

#We create the measure 'Concentration' by dividing Population/Area
df['Concentration']=df['Population']/df['Area']

#As you can see in the table above, coordinates are in degrees and in decimal and we need to extract latitude and longitude from the column Coordinates
df['Borough'] = df.apply(lambda row : row.Borough.split('[')[0],axis=1)
df['decimal']=df.apply(lambda row:row.Coordinates.split('/')[1],axis=1)
df['Lat']=df['decimal'].str[:17]
df['Lon']=df['decimal'].str[-8:]
df['lat_direction']=df.apply(lambda row:row.Lat.split('°')[1],axis=1)
df['lon_direction']=df.apply(lambda row:row.Lon.split('°')[1],axis=1)
df['Lat']=df.apply(lambda row:row.Lat.split('°')[0],axis=1)
df['Lon']=df.apply(lambda row:row.Lon.split('°')[0],axis=1)
df['Lat']=df['Lat'].str[-7:]

#We transform Lat and Lon to float in order to be able to operate with them as a number and not as a string
df = df.astype({"Lat":'float64'}) 
df = df.astype({"Lon":'float64'}) 
df = df.astype({"Concentration":'int64'}) 

#We specify that if the direction is West or South, the coordinates should be negative in order to display them correctly later
df['Lon'] = np.where(df['lon_direction']== 'W', df['Lon']*-1, df['Lon'])
df['Lat'] = np.where(df['lat_direction']== 'S', df['Lat']*-1, df['Lat'])

#We get rid of any columns that won't be needed
df=df.drop(['Coordinates','decimal','lat_direction','lon_direction'],axis=1)

df.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

,Borough,Population,Area,Concentration,Lat,Lon
0,Barking and Dagenham,212906,13.93,15283,51.5607,0.1557
1,Barnet,395896,33.49,11821,51.6252,-0.1517
2,Bexley,248287,23.38,10619,51.4549,0.1505
3,Brent,329771,16.70,19746,51.5588,-0.2817
4,Bromley,332336,57.97,5732,51.4039,0.0198


In [5]:
#We double check the data types are the ones we are looking for
df.dtypes

Borough           object
Population         int64
Area             float64
Concentration      int64
Lat              float64
Lon              float64
dtype: object

# 

We confirm we have the 32 boroughs in London

In [6]:
df.shape

(32, 6)

In order to simplify our data we will just focus on the Top-10 Boroughs in terms of concentration of population. We do this by sorting by Concentration

In [7]:
df.sort_values(['Concentration'], ascending=False,inplace=True)
df=df.head(10).reset_index()
df

,index,Borough,Population,Area,Concentration,Lat,Lon
0,28,Tower Hamlets,324745,7.63,42561,51.5099,-0.0059
1,17,Islington,242467,5.74,42241,51.5416,-0.1022
2,10,Hackney,281120,7.36,38195,51.5450,-0.0553
3,18,Kensington and Chelsea,156129,4.68,33360,51.5020,-0.1947
4,5,Camden,270029,8.40,32146,51.5290,-0.1255
5,31,Westminster,261317,8.29,31521,51.4973,-0.1372
6,20,Lambeth,326034,10.36,31470,51.4607,-0.1163
7,11,Hammersmith and Fulham,185143,6.33,29248,51.4927,-0.2339
8,26,Southwark,318830,11.14,28620,51.5035,-0.0804
9,23,Newham,353134,13.98,25259,51.5077,0.0469


We get the coordinates for the city of London

In [8]:
address = 'London, UK'

geolocator = Nominatim(user_agent="london_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of London are 51.5073219, -0.1276474.


In [9]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers and labels to map
for lat, lng, borough, concentration in zip(df['Lat'], df['Lon'], df['Borough'],df['Concentration']):
    label = '{}, {}'.format( borough, concentration)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  

map_london

### We use the Foursquare API to explore the Boroughs and segment them

In [10]:
CLIENT_ID = 'Hidden' 
CLIENT_SECRET = 'Hidden' 
VERSION = '20190605' 
LIMIT = 100 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: M2DUM5LSA0WK55EJNHBO3HQSJYRKLEYXACM4WPH1HSGA2Q1K
CLIENT_SECRET:VJZ4ADBTVME5OULPEUOQKU2JWZSQGUKWNUAXJ0UQEIEWMSOS


# Explore Boroughs in London

### I create a process to explore the venues in all Boroughs

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now I write the code to run the above function on each Borough and create a new dataframe called London_venues.

In [25]:
London_venues = getNearbyVenues(names=df['Borough'],
                                   latitudes=df['Lat'],
                                   longitudes=df['Lon']
                                  )

Tower Hamlets
Islington
Hackney
Kensington and Chelsea
Camden
Westminster
Lambeth
Hammersmith and Fulham 
Southwark
Newham


Size of the resulting dataframe

In [26]:
print(London_venues.shape)
London_venues.head()

(850, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Tower Hamlets,51.5099,-0.0059,Il Bianco Restaurant,51.506737,-0.006362,Italian Restaurant
1,Tower Hamlets,51.5099,-0.0059,Radisson Blu Edwardian,51.506850,-0.004154,Hotel
2,Tower Hamlets,51.5099,-0.0059,La Mook,51.506673,-0.005117,Coffee Shop
3,Tower Hamlets,51.5099,-0.0059,Billingsgate Market,51.506312,-0.014184,Fish Market
4,Tower Hamlets,51.5099,-0.0059,Maureen's Pie & Mash Shop,51.512950,-0.015261,English Restaurant


How many venues were returned by Borough

In [27]:
London_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Camden,100,100,100,100,100,100
Hackney,63,63,63,63,63,63
Hammersmith and Fulham,100,100,100,100,100,100
Islington,100,100,100,100,100,100
Kensington and Chelsea,100,100,100,100,100,100
Lambeth,100,100,100,100,100,100
Newham,38,38,38,38,38,38
Southwark,78,78,78,78,78,78
Tower Hamlets,71,71,71,71,71,71


#### Let's find out how many unique categories can be curated from all the returned venues

In [28]:
print('There are {} uniques categories.'.format(len(London_venues['Venue Category'].unique())))

There are 190 uniques categories.


# Borough Analysis

In [29]:
# one hot encoding
London_onehot = pd.get_dummies(London_venues[['Venue Category']], prefix="", prefix_sep="")

# add Borough column back to dataframe
London_onehot['Borough'] = London_venues['Borough'] 

# move Borough column to the first column
fixed_columns = [London_onehot.columns[-1]] + list(London_onehot.columns[:-1])
London_onehot = London_onehot[fixed_columns]

London_onehot.head()

,Borough,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,BBQ Joint,Badminton Court,Bakery,Bar,Beer Bar,Beer Store,Bike Shop,Bistro,Boarding House,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Camera Store,Canal,Canal Lock,Cantonese Restaurant,Caribbean Restaurant,Castle,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Creperie,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery,Dumpling Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Okonomiyaki Restaurant,Opera House,Organic Grocery,Outdoor Sculpture,Outlet Mall,Palace,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Road,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Smoothie Shop,Social Club,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Taxi Stand,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Trail,Train Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Windmill,Wine Bar,Wine Shop,Winery,Yoga Studio
0,Tower Hamlets,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Tower Hamlets,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Tower Hamlets,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Tower Hamlets,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [30]:
London_onehot.shape

(850, 191)

Group rows by Borough and by taking the mean of the frequency of occurrence of each category

In [31]:
London_grouped = London_onehot.groupby('Borough').mean().reset_index()
London_grouped

,Borough,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,BBQ Joint,Badminton Court,Bakery,Bar,Beer Bar,Beer Store,Bike Shop,Bistro,Boarding House,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Camera Store,Canal,Canal Lock,Cantonese Restaurant,Caribbean Restaurant,Castle,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Creperie,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery,Dumpling Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Okonomiyaki Restaurant,Opera House,Organic Grocery,Outdoor Sculpture,Outlet Mall,Palace,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Road,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Smoothie Shop,Social Club,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Taxi Stand,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Trail,Train Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Windmill,Wine Bar,Wine Shop,Winery,Yoga Studio
0,Camden,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.010000,0.020000,0.020000,0.000000,0.000000,0.00,0.00,0.040000,0.00,0.00,0.030000,0.000000,0.00,0.03,0.000000,0.000000,0.000000,0.000000,0.060000,0.00,0.01,0.01,0.00,0.00,0.000000,0.00,0.000000,0.010000,0.010000,0.010000,0.000000,0.020000,0.070000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.01,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.01,0.00,0.010000,0.000000,0.00,0.01,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.01,0.020000,0.00,0.01,0.01,0.00,0.010000,0.01,0.010000,0.000000,0.000000,0.030000,0.00,0.00,0.050000,0.000000,0.00,0.00,0.010000,0.00,0.010000,0.000000,0.00,0.000000,0.00,0.00,0.01,0.000000,0.00,0.000000,0.01,0.01,0.00,0.01,0.01,0.010000,0.000000,0.020000,0.020000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.030000,0.00,0.000000,0.00,0.000000,0.00,0.020000,0.00,0.020000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.00,0.010000,0.000000,0.02,0.000000,0.010000,0.01,0.000000,0.00,0.00,0.00,0.000000,0.010000,0.00,0.000000,0.000000,0.00,0.010000,0.020000,0.010000,0.000000,0.01,0.000000,0.00,0.000000,0.01,0.000000,0.01,0.000000,0.0200

In [32]:
London_grouped.shape

(10, 191)

Print each Borough with the top-5 most common venues

In [33]:
num_top_venues = 5

for bor in London_grouped['Borough']:
    print("----"+bor+"----")
    temp = London_grouped[London_grouped['Borough'] == bor].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Camden----
          venue  freq
0   Coffee Shop  0.07
1          Café  0.06
2         Hotel  0.05
3     Bookstore  0.04
4  Burger Joint  0.03


----Hackney----
         venue  freq
0  Coffee Shop  0.10
1          Pub  0.08
2         Café  0.08
3       Bakery  0.08
4      Brewery  0.06


----Hammersmith and Fulham ----
               venue  freq
0                Pub  0.14
1               Café  0.07
2        Coffee Shop  0.06
3  Indian Restaurant  0.06
4               Park  0.04


----Islington----
         venue  freq
0          Pub  0.11
1  Coffee Shop  0.07
2         Café  0.06
3         Park  0.04
4       Bakery  0.04


----Kensington and Chelsea----
                venue  freq
0              Garden  0.06
1  Italian Restaurant  0.05
2          Restaurant  0.04
3           Juice Bar  0.04
4               Hotel  0.04


----Lambeth----
                  venue  freq
0           Coffee Shop  0.07
1  Caribbean Restaurant  0.06
2           Pizza Place  0.05
3                   Pub  0.0

Put it into a dataframe and sort venues in desc order

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 5 venues for each neighborhood.

In [45]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Borough_venues_sorted = pd.DataFrame(columns=columns)
Borough_venues_sorted['Borough'] = London_grouped['Borough']

for ind in np.arange(London_grouped.shape[0]):
    Borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(London_grouped.iloc[ind, :], num_top_venues)

Borough_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Camden,Coffee Shop,Café,Hotel,Bookstore,Park
1,Hackney,Coffee Shop,Pub,Café,Bakery,Brewery
2,Hammersmith and Fulham,Pub,Café,Coffee Shop,Indian Restaurant,Park
3,Islington,Pub,Coffee Shop,Café,Park,Bakery
4,Kensington and Chelsea,Garden,Italian Restaurant,Café,Juice Bar,Restaurant
5,Lambeth,Coffee Shop,Caribbean Restaurant,Pub,Pizza Place,Cocktail Bar
6,Newham,Hotel,Airport Service,Coffee Shop,Sandwich Place,Bus Station
7,Southwark,Coffee Shop,Hotel,Park,Bakery,Garden
8,Tower Hamlets,Coffee Shop,Hotel,Park,Italian Restaurant,Sandwich Place
9,Westminster,Hotel,Coffee Shop,Sandwich Place,Café,Theater


# Cluster Boroughs

Run _k_-means to cluster the neighborhood into 3 clusters.

In [46]:
# set number of clusters
kclusters = 3

London_grouped_clustering = London_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(London_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 0, 0, 1, 0, 2, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood.

In [47]:
# add clustering labels
Borough_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

London_merged = df

# merge London_grouped with df to add latitude/longitude for each Borough
London_merged = London_merged.join(Borough_venues_sorted.set_index('Borough'), on='Borough')

London_merged

,index,Borough,Population,Area,Concentration,Lat,Lon,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,28,Tower Hamlets,324745,7.63,42561,51.5099,-0.0059,1,Coffee Shop,Hotel,Park,Italian Restaurant,Sandwich Place
1,17,Islington,242467,5.74,42241,51.5416,-0.1022,0,Pub,Coffee Shop,Café,Park,Bakery
2,10,Hackney,281120,7.36,38195,51.5450,-0.0553,0,Coffee Shop,Pub,Café,Bakery,Brewery
3,18,Kensington and Chelsea,156129,4.68,33360,51.5020,-0.1947,1,Garden,Italian Restaurant,Café,Juice Bar,Restaurant
4,5,Camden,270029,8.40,32146,51.5290,-0.1255,1,Coffee Shop,Café,Hotel,Bookstore,Park
5,31,Westminster,261317,8.29,31521,51.4973,-0.1372,1,Hotel,Coffee Shop,Sandwich Place,Café,Theater
6,20,Lambeth,326034,10.36,31470,51.4607,-0.1163,0,Coffee Shop,Caribbean Restaurant,Pub,Pizza Place,Cocktail Bar
7,11,Hammersmith and Fulham,185143,6.33,29248,51.4927,-0.2339,0,Pub,Café,Coffee Shop,Indian Restaurant,Park
8,26,Southwark,318830,11.14,28620,51.5035,-0.0804,1,Coffee Shop,Hotel,Park,Bakery,Garden
9,23,Newham,353134,13.98,25259,51.5077,0.0469,2,Hotel,Airport Service,Coffee Shop,Sandwich Place,Bus Station


In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(London_merged['Lat'], London_merged['Lon'], London_merged['Borough'], London_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

Cluster 1

In [52]:
London_merged.loc[London_merged['Cluster Labels'] == 0, London_merged.columns[[1] + list(range(4, London_merged.shape[1]))]]

,Borough,Concentration,Lat,Lon,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Islington,42241,51.5416,-0.1022,0,Pub,Coffee Shop,Café,Park,Bakery
2,Hackney,38195,51.5450,-0.0553,0,Coffee Shop,Pub,Café,Bakery,Brewery
6,Lambeth,31470,51.4607,-0.1163,0,Coffee Shop,Caribbean Restaurant,Pub,Pizza Place,Cocktail Bar
7,Hammersmith and Fulham,29248,51.4927,-0.2339,0,Pub,Café,Coffee Shop,Indian Restaurant,Park


Cluster 2

In [54]:
London_merged.loc[London_merged['Cluster Labels'] == 1, London_merged.columns[[1] + list(range(4, London_merged.shape[1]))]]

,Borough,Concentration,Lat,Lon,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Tower Hamlets,42561,51.5099,-0.0059,1,Coffee Shop,Hotel,Park,Italian Restaurant,Sandwich Place
3,Kensington and Chelsea,33360,51.5020,-0.1947,1,Garden,Italian Restaurant,Café,Juice Bar,Restaurant
4,Camden,32146,51.5290,-0.1255,1,Coffee Shop,Café,Hotel,Bookstore,Park
5,Westminster,31521,51.4973,-0.1372,1,Hotel,Coffee Shop,Sandwich Place,Café,Theater
8,Southwark,28620,51.5035,-0.0804,1,Coffee Shop,Hotel,Park,Bakery,Garden


Cluster 3

In [55]:
London_merged.loc[London_merged['Cluster Labels'] == 2, London_merged.columns[[1] + list(range(4, London_merged.shape[1]))]]

,Borough,Concentration,Lat,Lon,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
9,Newham,25259,51.5077,0.0469,2,Hotel,Airport Service,Coffee Shop,Sandwich Place,Bus Station
